# 1. 네이버 영화 리뷰 데이터에 대한 이해와 전처리

In [1]:
!pip3 install konlpy

     |████████████████████████████████| 19.4 MB 6.6 MB/s 
     |████████████████████████████████| 448 kB 51.7 MB/s 


In [2]:
import torch

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [4]:
import pandas as pd
import urllib.request
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tqdm import tqdm
import numpy as np

In [5]:
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

--2022-01-22 05:49:24--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘ratings_train.txt’

ratings_train.txt   100%[===================>]  13.95M  --.-KB/s    in 0.1s    

2022-01-22 05:49:26 (106 MB/s) - ‘ratings_train.txt’ saved [14628807/14628807]

--2022-01-22 05:49:26--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4893335 (4.7M) [text/plain]
Saving

('ratings_test.txt', <http.client.HTTPMessage at 0x7fb42757ab10>)

### 데이터 확인하기

- pd.read_table() 함수를 이용하여, ``ratings_train.txt``, ``ratings_test.txt`` 파일을 읽어와주세요. 변수명은 *train_data*, *test_data* 로 해주세요.

- 생성된 pandas data frame의 처음 10개의 행을 출력해주세요.

- 행의 길이를 출력해주세요

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
train_data = pd.read_table("/content/drive/MyDrive/NLP/네이버 리뷰 분석/data/movie_review/ratings_train.txt")
test_data = pd.read_table("/content/drive/MyDrive/NLP/네이버 리뷰 분석/data/movie_review/ratings_test.txt")

In [8]:
train_data.head(10)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1


In [9]:
test_data.head(10)

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
5,7898805,"음악이 주가 된, 최고의 음악영화",1
6,6315043,진정한 쓰레기,0
7,6097171,"마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다",0
8,8932678,갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한c...,0
9,6242223,"이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네..",1


In [10]:
len(train_data), len(test_data)

(150000, 50000)

## 전처리 단계

중복 제거, null값, 공백 제거, 한글과 공백 이외의 문자 제거 등을 수행합니다.

In [11]:
train_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
train_data['document'] = train_data['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경
train_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
train_data = train_data.dropna(how='any') # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(train_data))

전처리 후 테스트용 샘플의 개수 : 145393


In [12]:
test_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data['document'] = test_data['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경
test_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(test_data))

전처리 후 테스트용 샘플의 개수 : 48852


## 토큰화

In [13]:
import pickle

토큰화 작업은 시간이 꽤 걸립니다. 그래서 피클 파일들을 준비해 두었어요. 

In [14]:
START_FROM_SCRATCH = False

stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

if START_FROM_SCRATCH:
    # 10분쯤 걸려요..!
    okt = Okt()
    X_train = []
    for sentence in tqdm(train_data['document']):
        tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
        stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
        X_train.append(stopwords_removed_sentence)
    X_test = []
    for sentence in tqdm(test_data['document']):
        tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
        stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
        X_test.append(stopwords_removed_sentence)
    
    # X_train, X_test 객체 저장하기
    with open("X_train.pkl", "wb") as f:
        pickle.dump(X_train, f)
    with open("X_test.pkl", "wb") as f:
        pickle.dump(X_test, f)

# X_train, X_test 객체 불러오기
with open("/content/drive/MyDrive/NLP/네이버 리뷰 분석/data/movie_review/X_train.pkl", "rb") as f:
    X_train = pickle.load(f)
with open("/content/drive/MyDrive/NLP/네이버 리뷰 분석/data/movie_review/X_test.pkl", "rb") as f:
    X_test = pickle.load(f)

### Word to index 생성하기

In [15]:
from collections import defaultdict

In [16]:
word_count = defaultdict(int)

for tokens in tqdm(X_train):
    for token in tokens:
        word_count[token] += 1

word_count = sorted(word_count.items(), key=lambda x: x[1], reverse=True)

100%|██████████| 145393/145393 [00:00<00:00, 258040.60it/s]


모든 단어를 다 쓰지 말고, 빈도가 가장 높은 20000개의 단어만 써봅시다. 

In [17]:
vocab_size = 20000
word_count = word_count[:vocab_size]
word_count

[('영화', 50367),
 ('보다', 41072),
 ('을', 23208),
 ('없다', 15572),
 ('이다', 13196),
 ('있다', 12429),
 ('좋다', 12001),
 ('너무', 11124),
 ('다', 10105),
 ('정말', 9720),
 ('되다', 8852),
 ('재밌다', 8802),
 ('적', 8747),
 ('만', 8659),
 ('같다', 8343),
 ('진짜', 8312),
 ('로', 7834),
 ('아니다', 7753),
 ('않다', 7722),
 ('점', 7547),
 ('에서', 6981),
 ('만들다', 6507),
 ('나오다', 6397),
 ('연기', 6326),
 ('것', 6286),
 ('평점', 6266),
 ('내', 5919),
 ('최고', 5785),
 ('그', 5670),
 ('나', 5563),
 ('안', 5495),
 ('인', 5469),
 ('스토리', 5332),
 ('생각', 5313),
 ('못', 5295),
 ('왜', 5120),
 ('드라마', 5044),
 ('게', 4928),
 ('감동', 4844),
 ('사람', 4836),
 ('보고', 4658),
 ('이렇다', 4590),
 ('말', 4527),
 ('고', 4501),
 ('아깝다', 4419),
 ('더', 4352),
 ('배우', 4286),
 ('때', 4279),
 ('ㅋㅋ', 4276),
 ('아', 4138),
 ('감독', 4114),
 ('거', 4081),
 ('그냥', 4062),
 ('재미있다', 4038),
 ('요', 4029),
 ('재미', 3879),
 ('시간', 3866),
 ('내용', 3813),
 ('뭐', 3778),
 ('까지', 3777),
 ('중', 3728),
 ('주다', 3724),
 ('자다', 3626),
 ('하고', 3563),
 ('지루하다', 3508),
 ('재미없다', 3480),
 ('쓰레기', 34

UNK, PAD 토큰을 추가해 봅시다.

In [18]:
pad = 0
unk = 1

pad_token = "[PAD]"
unk_token = "[UNK]"

In [19]:
word_count.insert(pad, (pad_token,-1))
word_count.insert(unk, (unk_token,-1))
vocab_size += 2
print(list(word_count))

[('[PAD]', -1), ('[UNK]', -1), ('영화', 50367), ('보다', 41072), ('을', 23208), ('없다', 15572), ('이다', 13196), ('있다', 12429), ('좋다', 12001), ('너무', 11124), ('다', 10105), ('정말', 9720), ('되다', 8852), ('재밌다', 8802), ('적', 8747), ('만', 8659), ('같다', 8343), ('진짜', 8312), ('로', 7834), ('아니다', 7753), ('않다', 7722), ('점', 7547), ('에서', 6981), ('만들다', 6507), ('나오다', 6397), ('연기', 6326), ('것', 6286), ('평점', 6266), ('내', 5919), ('최고', 5785), ('그', 5670), ('나', 5563), ('안', 5495), ('인', 5469), ('스토리', 5332), ('생각', 5313), ('못', 5295), ('왜', 5120), ('드라마', 5044), ('게', 4928), ('감동', 4844), ('사람', 4836), ('보고', 4658), ('이렇다', 4590), ('말', 4527), ('고', 4501), ('아깝다', 4419), ('더', 4352), ('배우', 4286), ('때', 4279), ('ㅋㅋ', 4276), ('아', 4138), ('감독', 4114), ('거', 4081), ('그냥', 4062), ('재미있다', 4038), ('요', 4029), ('재미', 3879), ('시간', 3866), ('내용', 3813), ('뭐', 3778), ('까지', 3777), ('중', 3728), ('주다', 3724), ('자다', 3626), ('하고', 3563), ('지루하다', 3508), ('재미없다', 3480), ('쓰레기', 3434), ('수', 3433), ('네', 3420), ('모르다

w2i 딕셔너리를 정의해봅시다.

In [20]:
w2i = {} 
for pair in tqdm(word_count):
    if pair[0] not in w2i:
        w2i[pair[0]] = len(w2i)
print(w2i)

100%|██████████| 20002/20002 [00:00<00:00, 1196799.79it/s]

{'[PAD]': 0, '[UNK]': 1, '영화': 2, '보다': 3, '을': 4, '없다': 5, '이다': 6, '있다': 7, '좋다': 8, '너무': 9, '다': 10, '정말': 11, '되다': 12, '재밌다': 13, '적': 14, '만': 15, '같다': 16, '진짜': 17, '로': 18, '아니다': 19, '않다': 20, '점': 21, '에서': 22, '만들다': 23, '나오다': 24, '연기': 25, '것': 26, '평점': 27, '내': 28, '최고': 29, '그': 30, '나': 31, '안': 32, '인': 33, '스토리': 34, '생각': 35, '못': 36, '왜': 37, '드라마': 38, '게': 39, '감동': 40, '사람': 41, '보고': 42, '이렇다': 43, '말': 44, '고': 45, '아깝다': 46, '더': 47, '배우': 48, '때': 49, 'ㅋㅋ': 50, '아': 51, '감독': 52, '거': 53, '그냥': 54, '재미있다': 55, '요': 56, '재미': 57, '시간': 58, '내용': 59, '뭐': 60, '까지': 61, '중': 62, '주다': 63, '자다': 64, '하고': 65, '지루하다': 66, '재미없다': 67, '쓰레기': 68, '수': 69, '네': 70, '모르다': 71, '가다': 72, '들다': 73, '그렇다': 74, '싶다': 75, '지': 76, '작품': 77, '사랑': 78, '알다': 79, '하나': 80, '다시': 81, '마지막': 82, '볼': 83, '이건': 84, '정도': 85, '저': 86, '완전': 87, '오다': 88, 'ㅋ': 89, 'ㅠㅠ': 90, '많다': 91, '처음': 92, 'ㅋㅋㅋ': 93, '장면': 94, '액션': 95, '주인공': 96, '이렇게': 97, '안되다': 98, '걸': 99, '차다': 100, '

In [21]:
keys, values = list(w2i.keys()), list(w2i.values())
keys[0], values[0]

('[PAD]', 0)

## 데이터 준비하기

데이터로더를 쓰기 전, 연습 먼저 해봅시다.

In [22]:
import torch
from torch.utils.data import Dataset, DataLoader

쉬운 데이터 처리를 위해, 문장의 길이를 앞에서부터 30 tokens로 제한합시다.

string으로 되어있는 데이터를 w2i dictionary를 활용하여 정수로 바꾸어 줍시다. 

w2i dictionary에 없다면, 어떻게 해야 할까요?


In [23]:
max_length = 30

In [24]:
X_train_int = []
for sentence in tqdm(X_train):
    sentence[:max_length]
    sentence_int = list(map(lambda w:w2i[w] if w in w2i else w2i['[UNK]'], sentence))
    X_train_int.append(torch.LongTensor(sentence_int))

100%|██████████| 145393/145393 [00:01<00:00, 74229.92it/s]


In [25]:
X_train_int[0]

tensor([ 51, 455,  17, 261, 660])

torch의 pad_sequence를 활용하여, padding 을 해 봅시다. 

``batch_first`` 와 ``padding_value`` argument들을 활용해보세요. 

In [26]:
from torch.nn.utils.rnn import pad_sequence

In [27]:
X_train_pad = pad_sequence(X_train_int, batch_first = True, padding_value = pad)

In [28]:
X_train_pad[0], train_data["document"][0]

(tensor([ 51, 455,  17, 261, 660,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0]), '아 더빙 진짜 짜증나네요 목소리')

라벨(y) 도 같이 준비합시다.

In [29]:
y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

In [30]:
len(X_train), len(y_train), len(X_test), len(y_test)

(145393, 145393, 48852, 48852)

In [31]:
class MovieReviewDataset(Dataset):
    def __init__(self, X, y, max_length = max_length, pad = pad):
        self.X = self.preprocess_data(X, max_length, pad = pad)
        self.y = torch.Tensor(y)
        
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

    def preprocess_data(self, X, max_length = max_length, pad = pad):
        data = []

        #1. 최대 길이보다 길면 자르기
        for sentence in tqdm(X):
            sentence = sentence[:max_length]
            sentence_int = list(map(lambda w:w2i[w] if w in w2i else w2i['[UNK]'], sentence))
            data.append(torch.LongTensor(sentence_int))
        #2. padding
        data = pad_sequence(data, batch_first = True, padding_value = pad)
        #3. 필요하다면, 텐서 타입으로 변경
        
        return data

In [32]:
datasets = {}
datasets["train"] = MovieReviewDataset(X_train, y_train)
datasets["test"] = MovieReviewDataset(X_test, y_test)

100%|██████████| 48852/48852 [00:00<00:00, 68157.28it/s]


In [33]:
datasets["train"][0]

(tensor([ 51, 455,  17, 261, 660,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0]), tensor(0.))

In [34]:
dataloaders = {}
dataloaders["train"]= DataLoader(datasets["train"], batch_size=64, shuffle=True)
dataloaders["test"]= DataLoader(datasets["test"], batch_size=64, shuffle=True)

In [35]:
next(iter(dataloaders["train"]))

[tensor([[ 153,   58,  815,  ...,    0,    0,    0],
         [8232, 5559, 9962,  ...,    0,    0,    0],
         [  29, 2617,   61,  ...,    0,    0,    0],
         ...,
         [  50,  101,  184,  ...,    0,    0,    0],
         [   5,   99,  150,  ...,    0,    0,    0],
         [1138,   28,   86,  ...,    0,    0,    0]]),
 tensor([1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0.,
         1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1.,
         0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1.,
         1., 1., 0., 1., 0., 0., 0., 1., 0., 1.])]

# 2. LSTM으로 네이버 영화 리뷰 감성 분류하기
* Rnn에서 Cell state부분이 추가된다.
* LSTM의 Cell state의 gate는 forget,input,ouput이 존재한다.

모델을 구성하고, device에 올려봅니다. 

In [64]:
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers = 1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(input_size = embedding_dim, hidden_size = hidden_size, num_layers = num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, 1)

    def forward(self, x):
        embeddings = self.embedding(x)
        output, (hidden, cell) = self.rnn(embeddings)
        final_output = torch.sigmoid(self.linear(output[:, -1, :]))
        return final_output

In [65]:
model = LSTM(vocab_size = vocab_size, embedding_dim = 100, hidden_size = 128, num_layers = 1)
model.to(device)

LSTM(
  (embedding): Embedding(20002, 100)
  (rnn): LSTM(100, 128, batch_first=True)
  (linear): Linear(in_features=128, out_features=1, bias=True)
)

In [66]:
sample_x, sample_y = next(iter(dataloaders["train"]))

In [67]:
sample_x.shape, sample_y.shape

(torch.Size([64, 30]), torch.Size([64]))

In [68]:
len(dataloaders["train"])

2272

모델을 training 시켜봅시다.

In [69]:
learning_rate=1e-3
epochs = 10
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
criterion = nn.BCELoss()

model.train()
for e in range(1, epochs+1):
    train_loss = 0.0
    for x,y in dataloaders["train"]:
        # 1. forward
        x, y = x.to(device), y.to(device).unsqueeze(1)
        optimizer.zero_grad()
        y_pred = model(x)

        # 2. backward
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        
    epoch_loss = train_loss / len(dataloaders["train"])
    print(f"EPOCH {e}: {epoch_loss}")

EPOCH 1: 0.47725964264130927
EPOCH 2: 0.32848056313962165
EPOCH 3: 0.2754196310448657
EPOCH 4: 0.22641223986324868
EPOCH 5: 0.17741129119408278
EPOCH 6: 0.13497965410269772
EPOCH 7: 0.10214765713112363
EPOCH 8: 0.08027324885884989
EPOCH 9: 0.06500952917870141
EPOCH 10: 0.055387314624734334


test 데이터셋에서 정확도를 산출해봅시다.

In [70]:
with torch.no_grad():
    test_loss = 0.0
    test_correct = 0
    test_total = 0
    for x,y in dataloaders["test"]:

    # 1. 모델 아웃풋 산출
        x, y = x.to(device), y.to(device).unsqueeze(1)
        y_pred = torch.round(model(x))
    
    # 2. 맞은 개수, 총 개수 업데이트
        test_correct += torch.sum(y_pred == y)
        test_total += len(y)
    
        loss = criterion(y_pred, y)
        test_loss += loss.item()

print(f"accuracy: {test_correct/test_total}")

accuracy: 0.8367109298706055


# 3. RNN 사용해보기

In [71]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers = 1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(input_size = embedding_dim, hidden_size = hidden_size, num_layers = num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, 1)

    def forward(self, x):
        embeddings = self.embedding(x)
        output, hidden= self.rnn(embeddings)
        final_output = torch.sigmoid(self.linear(output[:, -1, :]))
        return final_output

In [72]:
model2 = RNN(vocab_size = vocab_size, embedding_dim = 100, hidden_size = 128, num_layers = 1)
model2.to(device)

RNN(
  (embedding): Embedding(20002, 100)
  (rnn): RNN(100, 128, batch_first=True)
  (linear): Linear(in_features=128, out_features=1, bias=True)
)

In [73]:
learning_rate=1e-3
epochs = 10
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
criterion = nn.BCELoss()

model2.train()
for e in range(1, epochs+1):
    train_loss = 0.0
    for x,y in dataloaders["train"]:
        # 1. forward
        x, y = x.to(device), y.to(device).unsqueeze(1)
        optimizer.zero_grad()
        y_pred = model2(x)

        # 2. backward
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        
    epoch_loss = train_loss / len(dataloaders["train"])
    print(f"EPOCH {e}: {epoch_loss}")

EPOCH 1: 0.7274092754585223
EPOCH 2: 0.727411051188976
EPOCH 3: 0.7274031242959096
EPOCH 4: 0.7274020329957277
EPOCH 5: 0.7274073110762197
EPOCH 6: 0.727406503711368
EPOCH 7: 0.727405871172816
EPOCH 8: 0.7274010829679983
EPOCH 9: 0.7273979650836595
EPOCH 10: 0.7274013103418787


In [74]:
sample_x, sample_y = next(iter(dataloaders["train"]))
print(sample_x.shape, sample_y.shape)
print(len(dataloaders["train"]))

torch.Size([64, 30]) torch.Size([64])
2272


테스트 데이터로 정확도 산출!

In [75]:
with torch.no_grad():
    test_loss = 0.0
    test_correct = 0
    test_total = 0
    for x,y in dataloaders["test"]:

    # 1. 모델 아웃풋 산출
        x, y = x.to(device), y.to(device).unsqueeze(1)
        y_pred = torch.round(model2(x))
    
    # 2. 맞은 개수, 총 개수 업데이트
        test_correct += torch.sum(y_pred == y)
        test_total += len(y)
    
        loss = criterion(y_pred, y)
        test_loss += loss.item()

print(f"accuracy: {test_correct/test_total}")

accuracy: 0.4970523416996002


# 4. GRU로 사용해보기
* RNN과 전체적으로 비슷하면 Hidden state에는 2개의 gate가 사용된다.

In [77]:
import torch.nn as nn

class GRU(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers = 1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.GRU(input_size = embedding_dim, hidden_size = hidden_size, num_layers = num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, 1)

    def forward(self, x):
        embeddings = self.embedding(x)
        output, hidden= self.rnn(embeddings)
        final_output = torch.sigmoid(self.linear(output[:, -1, :]))
        return final_output

In [78]:
model3 = GRU(vocab_size = vocab_size, embedding_dim = 100, hidden_size = 128, num_layers = 1)
model3.to(device)

GRU(
  (embedding): Embedding(20002, 100)
  (rnn): GRU(100, 128, batch_first=True)
  (linear): Linear(in_features=128, out_features=1, bias=True)
)

In [79]:
learning_rate=1e-3
epochs = 10
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
criterion = nn.BCELoss()

model3.train()
for e in range(1, epochs+1):
    train_loss = 0.0
    for x,y in dataloaders["train"]:
        # 1. forward
        x, y = x.to(device), y.to(device).unsqueeze(1)
        optimizer.zero_grad()
        y_pred = model3(x)

        # 2. backward
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        
    epoch_loss = train_loss / len(dataloaders["train"])
    print(f"EPOCH {e}: {epoch_loss}")

EPOCH 1: 0.701905002843746
EPOCH 2: 0.7019068975473793
EPOCH 3: 0.7019074060232706
EPOCH 4: 0.7019090838205646
EPOCH 5: 0.7019071160802539
EPOCH 6: 0.7019108058228879
EPOCH 7: 0.7019038618817716
EPOCH 8: 0.701907385953925
EPOCH 9: 0.7019061088614481
EPOCH 10: 0.7019041254854118


In [80]:
sample_x, sample_y = next(iter(dataloaders["train"]))
print(sample_x.shape, sample_y.shape)
print(len(dataloaders["train"]))

torch.Size([64, 30]) torch.Size([64])
2272


In [81]:
with torch.no_grad():
    test_loss = 0.0
    test_correct = 0
    test_total = 0
    for x,y in dataloaders["test"]:

    # 1. 모델 아웃풋 산출
        x, y = x.to(device), y.to(device).unsqueeze(1)
        y_pred = torch.round(model3(x))
    
    # 2. 맞은 개수, 총 개수 업데이트
        test_correct += torch.sum(y_pred == y)
        test_total += len(y)
    
        loss = criterion(y_pred, y)
        test_loss += loss.item()

print(f"accuracy: {test_correct/test_total}")

accuracy: 0.4936133921146393


# 리뷰 예측해보기

LSTM이 정확도가 제일 높으므로 LSTM 모델로 사용!

In [44]:
def sentiment_predict(new_sentence):
    print(new_sentence)

  # 전처리 과정
    okt = Okt()
    new_sentence = new_sentence.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
    new_sentence = new_sentence.replace('^ +', "")
    new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    new_sentence = new_sentence[:max_length]
    new_sentence = list(map(lambda w:w2i[w] if w in w2i else w2i['[UNK]'], new_sentence))
    new_sentence = torch.LongTensor(new_sentence).to(device).unsqueeze(0)
  
    print(f"tokenized: {new_sentence}")

    score = float(model(new_sentence))
    if(score > 0.5):
        print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
    else:
        print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

In [45]:
sentiment_predict('이 영화 개꿀잼 ㅋㅋㅋ')

이 영화 개꿀잼 ㅋㅋㅋ
tokenized: tensor([[   2, 3141,   93]], device='cuda:0')
99.62% 확률로 긍정 리뷰입니다.



In [46]:
sentiment_predict('이 영화 핵노잼 ㅠㅠ')

이 영화 핵노잼 ㅠㅠ
tokenized: tensor([[   2, 1280,  348,   90]], device='cuda:0')
99.91% 확률로 부정 리뷰입니다.



In [47]:
sentiment_predict('이딴게 영화냐 ㅉㅉ')

이딴게 영화냐 ㅉㅉ
tokenized: tensor([[433,  39,   2, 221, 654]], device='cuda:0')
99.58% 확률로 부정 리뷰입니다.



In [48]:
sentiment_predict('감독 뭐하는 놈이냐?')

감독 뭐하는 놈이냐?
tokenized: tensor([[ 52,  60, 351, 806,   1]], device='cuda:0')
99.69% 확률로 부정 리뷰입니다.



In [84]:
sentiment_predict('와 역대급으로 따봉 주고 싶어')

와 역대급으로 따봉 주고 싶어
tokenized: tensor([[  88,  797,  188,  399, 3719,   63,   75]], device='cuda:0')
61.51% 확률로 긍정 리뷰입니다.



In [82]:
sentiment_predict('이 영화 재밌다')

이 영화 재밌다
tokenized: tensor([[ 2, 13]], device='cuda:0')
82.94% 확률로 긍정 리뷰입니다.

